<a href="https://colab.research.google.com/github/EzeGamer135/UCC-Research-Laboratories/blob/main/Investigaciones/modelos/Open_Skies_SUNBURST_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import math

theta = np.radians(65.3)  # ángulo de lanzamiento en rad
m = 500                   # masa del carguero en kg
v = 944                   # velocidad de lanzamiento en m/s
r = 12                    # radio de la centrifugadora en m
g = 9.81                  # gravedad ASL en m/s²
R_E = 6378000             # radio de la Tierra en m
rho0 = 1.225              # densidad del aire ASL en kg/m³
H = 8500                  # escala de altura de la atmósfera en m
Cd = 0.47                 # coeficiente de arrastre
A = 1.0                   # área frontal del carguero en m²

In [ ]:
E_cinetica = 0.5 * m * v**2

# Velocidad angular en rad/s
omega = math.sqrt((2 * E_cinetica) / (m * r**2))
omega_deg_per_s = omega * (180 / math.pi)

print("J: " + str(E_cinetica) + ", °/s: " + str(omega_deg_per_s) + ", RPM: " + str(omega_deg_per_s/6) + ", RPS: " + str(omega_deg_per_s/360))


Ejecutar este código sólo cuando se hayan definido las constantes en el código 1.

In [ ]:
# Descomponer la velocidad inicial en componentes
vx = v * np.cos(theta)
vy = v * np.sin(theta)
x = 0.0           # Posición horizontal inicial (m)
y = 0.0           # Altitud inicial (m)
t = 0.0           # Tiempo inicial (s)

# Parámetros de integración
dt = 0.1          # Paso de tiempo (s)
t_max = 300       # Tiempo máximo de simulación (s)

# Listas para almacenar los datos simulados
t_values = []
x_values = []
y_values = []
v_values = []

# Bucle de simulación (método de Euler)
while t < t_max and y >= 0:
    t_values.append(t)
    x_values.append(x)
    y_values.append(y)
    current_speed = np.sqrt(vx**2 + vy**2)
    v_values.append(current_speed)

    # Actualizar la densidad del aire en función de la altitud
    current_rho = rho0 * np.exp(-y / H)

    # Fuerza de arrastre
    F_drag = 0.5 * Cd * A * current_rho * current_speed**2
    a_drag = F_drag / m  # aceleración debido al drag (módulo)

    # Descomponer la aceleración de drag en componentes (dirección opuesta a la velocidad)
    ax_drag = - a_drag * (vx / current_speed) if current_speed != 0 else 0
    ay_drag = - a_drag * (vy / current_speed) if current_speed != 0 else 0

    # Aceleración total: en X, sólo drag; en Y: gravedad (constante) + drag
    ax = ax_drag
    ay = -g + ay_drag  # gravedad actúa hacia abajo

    # Actualización de velocidades (método de Euler)
    vx += ax * dt
    vy += ay * dt

    # Actualización de posiciones
    x += vx * dt
    y += vy * dt

    t += dt

# Convertir las listas a arrays
t_values = np.array(t_values)
x_values = np.array(x_values)
y_values = np.array(y_values)
v_values = np.array(v_values)

# Graficar Altitud y Velocidad vs. Tiempo
fig, ax1 = plt.subplots(figsize=(10, 6))
ax1.set_xlabel("Tiempo (s)")
ax1.set_ylabel("Altitud (km)", color="tab:red")
ax1.plot(t_values, y_values/1000, 'r-', label="Altitud")
ax1.tick_params(axis="y", labelcolor="tab:red")
ax1.set_ylim(0, 150)  # hasta 150 km

ax1b = ax1.twinx()
ax1b.set_ylabel("Velocidad (m/s)", color="tab:blue")
ax1b.plot(t_values, v_values, 'b--', label="Velocidad")
ax1b.tick_params(axis="y", labelcolor="tab:blue")
if len(v_values) > 0:
    ax1b.set_ylim(0, max(v_values)*1.1)

plt.title("Altitud y Velocidad vs. Tiempo")
plt.tight_layout()
plt.show()

# Graficar Altitud vs. Longitud (perfil de vuelo)
fig, ax2 = plt.subplots(figsize=(10, 6))
ax2.set_xlabel("Distancia Horizontal (km)")
ax2.set_ylabel("Altitud (km)")
ax2.plot(x_values/1000, y_values/1000, 'g-', label="Trayectoria")
ax2.set_title("Perfil de Vuelo: Altitud vs. Longitud")
ax2.set_xlim(0, max(x_values)/1000*1.1)
ax2.set_ylim(0, max(y_values)/1000*1.1)
plt.tight_layout()
plt.show()

# Extraer el índice del apogeo (máxima altitud) y mostrar resultados
if len(y_values) > 0:
    apogeo_idx = np.argmax(y_values)
    print("Altitud máxima alcanzada: {:.2f} km".format(y_values[apogeo_idx] / 1000))
    print("Tiempo en el apogeo: {:.2f} s".format(t_values[apogeo_idx]))
    print("Velocidad en el apogeo: {:.2f} m/s".format(v_values[apogeo_idx]))
else:
    print("No se generaron datos de la simulación.")